<a href="https://colab.research.google.com/github/aidan-lew/junk-drawer/blob/main/LLM_scanner_chat_gpt_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pandas numpy scikit-learn beautifulsoup4 requests pymongo finrl
!pip install git+https://github.com/AI4Finance-Foundation/FinGPT.git
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install gymnasium
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 18.5 MB/s eta 0:00:00
  Cloning https://github.com/AI4Finance-Foundation/FinGPT.git to /tmp/pip-req-build-yn3rv83c
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinGPT.git /tmp/pip-req-build-yn3rv83c
  Resolved https://github.com/AI4Finance-Foundation/FinGPT.git to commit 7a7c76cdb71f743905a4d0eb373efcf447413e87
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 18.3 MB/s eta 0:00:00
  Created wheel for FinGPT: filename=FinGPT-0.0.1-py3-none-any.whl size=39326 sha256=28a595c713bf1ad823e982f62f90d753330b3f8ee

In [ ]:
!pip install stable-baselines3
!pip install alpaca-trade-api
!pip install exchange_calendars
!pip install stockstats
!pip install wrds
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from transformers import pipeline
from finrl.agents.stablebaselines3.models import DRLAgent
#from finrl.marketdata.yahoodownloader import YahooDownloader
#from finrl.preprocessing.preprocessors import FeatureEngineer
#from finrl.config import config
from pymongo import MongoClient
import openai


In [ ]:
def fetch_news(api_key, query='stock market', page_size=1):
    url = f'https://newsapi.org/v2/everything?q={query}&pageSize={page_size}&apiKey={api_key}'
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes
    articles = response.json().get('articles', [])
    return articles

def process_articles(articles):
    data = []
    for article in articles:
        title = article.get('title', '')
        #print(title)
        description = article.get('description', '')
        #print(description)
        content = f"{title}. {description}"
        #print(content)
        data.append({'title': title, 'description': description, 'content': content})

    return pd.DataFrame(data)

news_api_key = ''
news_articles = fetch_news(news_api_key)
processed_data = process_articles(news_articles)

#processed_data['title']


######## giving some weird error, but it is a DF - works in Jupyter

GameStop Stock Soars Again After New Roaring Kitty Post
GameStop stock skyrocketed more than 100% on Monday before giving back just some of those huge gains, the latest meme stock rally fueled by the investor and social media personality known as “Roaring Kitty.”Read more...
GameStop Stock Soars Again After New Roaring Kitty Post. GameStop stock skyrocketed more than 100% on Monday before giving back just some of those huge gains, the latest meme stock rally fueled by the investor and social media personality known as “Roaring Kitty.”Read more...


0    GameStop Stock Soars Again After New Roaring K...
Name: title, dtype: object

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from fingpt import FinGPT
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

#loadd fingpt llama-7B
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
gpt_model = gpt_model.eval()


def analyze_with_fingpt(processed_data):
    predictions = []
    for _, row in processed_data.iterrows():
        prompt = f"Based on the following news article, predict which specific stocks or financial sectors are likely to significantly grow: {row['content']}"
        inputs = tokenizer(prompt, return_tensors='pt')
        outputs = gpt_model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(prediction)
    processed_data['fingpt_predictions'] = predictions
    return processed_data

analyzed_data = analyze_with_fingpt(processed_data)
print(analyzed_data.head())


In [ ]:
# Load and process historical stock data
def load_stock_data(tickers):
    df = YahooDownloader(start_date='2020-01-01',
                         end_date='2023-12-31',
                         ticker_list=tickers).fetch_data()
    fe = FeatureEngineer(use_technical_indicator=True,
                         tech_indicator_list=config.TECHNICAL_INDICATORS_LIST,
                         use_turbulence=True,
                         user_defined_feature=False)
    processed = fe.preprocess_data(df)
    return processed

# Example tickers
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN']
stock_data = load_stock_data(tickers)

# Train FinRL model
def train_finrl_model(processed_data):
    env = StockTradingEnv(df=processed_data, initial_amount=1000000, hmax=100, turbulence_threshold=None,
                          risk_indicator_col='turbulence')
    agent = DRLAgent(env=env)
    model = agent.get_model("a2c")
    trained_model = agent.train_model(model=model, tb_log_name='a2c', total_timesteps=5000)
    return trained_model

trained_model = train_finrl_model(stock_data)

# Make predictions
def make_predictions(model, processed_data):
    env = StockTradingEnv(df=processed_data, initial_amount=1000000, hmax=100, turbulence_threshold=None,
                          risk_indicator_col='turbulence')
    predictions = DRLAgent.DRL_prediction(model=model, environment=env)
    return predictions

predictions = make_predictions(trained_model, stock_data)
print(predictions)


In [ ]:
def store_data_to_mongodb(data, db_name='stock_news', collection_name='predictions'):
    client = MongoClient('mongodb://localhost:27017/')
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(data.to_dict('records'))

store_data_to_mongodb(analyzed_data)
